# Practice Session 08: Data streams
<font size="+2" color="blue">Additional results: Heap's law</font>

Author: <font color="blue">Aniol Petit Cabarrocas</font>

E-mail: <font color="blue">aniol.petit01@estudiant.upf.edu</font>

Date: <font color="blue">20/11/2024</font>

In [ ]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

In [ ]:
# Leave this code as-is

INPUT_FILE = "movie_lines.tsv.gz"

In [ ]:
# Leave this code as-is

POS_NOUN = 'NN'
POS_VERB = 'VB'
POS_ADJECTIVE = 'JJ'

# Producer in Python that reads a file by words that are nouns
def read_by_parts_of_speech(filename, parts_of_speech, max_words=-1, report_every=-1):
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for sentence in nltk.sent_tokenize(text):
                
                tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
                for word in [part[0] for part in tagged if part[1] in parts_of_speech]:
                
                    counter += 1

                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [ ]:
import time
start_time = time.time()
for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words=20000, report_every=5000):
    # Prints 1/1000 of words
    if random.random() < 0.001:
        print("Current noun '%s'" % (word)) 
end_time = time.time()
print(f"Total time: {end_time - start_time}")

# 1. Determine approximately the top-10 words

<font size="+1" color="red">Replace this cell with your code for "add_reservoir"</font>

In [ ]:
def add_to_reservoir(reservoir, item, max_reservoir_size):
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    else:
        index = random.randint(0, len(reservoir))
        if index < max_reservoir_size:
            reservoir[index] = item
    assert len(reservoir) <= max_reservoir_size

<font size="+1" color="red">Replace this cell with your code for "reservoir_sampling"</font>

In [ ]:
def reservoir_sampling(filename, parts_of_speech, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    
    words_read = 0

    for word in read_by_parts_of_speech(filename, parts_of_speech, max_words=max_words, report_every=report_every):
        words_read += 1

        if max_words != -1 and words_read > max_words:
            break

        if len(reservoir) < reservoir_size:
            add_to_reservoir(reservoir, word, reservoir_size)
        else:
            if random.random() < reservoir_size / words_read:
                add_to_reservoir(reservoir, word, reservoir_size)

    return (words_read, reservoir)

In [ ]:
# Leave this code as-is

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=30000, report_every=10000)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

In [ ]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:20]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

<font size="+1" color="red">Replace this cell with your code to print the top items and their relative frequencies</font>

In [ ]:
freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

total_items = len(reservoir)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:20]

for absolute_frequency, word in most_frequent_items:
    relative_frequency = (absolute_frequency / total_items) * 100  
    print("%d %s (%.2f%%)" % (absolute_frequency, word, relative_frequency))

<font size="+1" color="red">Increase the max limit of words so that one pass takes about 2-3 minutes to be completed. Replace this cell with your code to try different reservoir sizes. In each case, print your estimate for the relative and absolute frequency of the words in the entire dataset.</font>

In [ ]:
reservoir_sizes = [50, 100, 500, 1000, 1500]
max_words = 20000  

dataset_size = max_words  

for reservoir_size in reservoir_sizes:
    print(f"\nReservoir Size: {reservoir_size}")
    
    items_seen, reservoir = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=max_words, report_every=20000)
    
    freq = {}
    for item in reservoir:
        freq[item] = freq.get(item, 0) + 1

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    
    for absolute_frequency, word in most_frequent_items:
        estimated_absolute_frequency = absolute_frequency * dataset_size / reservoir_size
        estimated_relative_frequency = (estimated_absolute_frequency / dataset_size) * 100
        print(f"{word}: Estimated Absolute Frequency = {estimated_absolute_frequency:.0f}, "
              f"Estimated Relative Frequency = {estimated_relative_frequency:.2f}%")

In [ ]:
reservoir_sizes = [50, 100, 500, 1000, 1500]
max_words = 20000  

dataset_size = max_words

for reservoir_size in reservoir_sizes:
    print(f"\nReservoir Size: {reservoir_size}")
    
    items_seen, reservoir = reservoir_sampling(INPUT_FILE, [POS_ADJECTIVE], reservoir_size, max_words=max_words, report_every=20000)
    
    freq = {}
    for item in reservoir:
        freq[item] = freq.get(item, 0) + 1

    most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:5]
    
    for absolute_frequency, word in most_frequent_items:
        estimated_absolute_frequency = absolute_frequency * dataset_size / reservoir_size
        estimated_relative_frequency = (estimated_absolute_frequency / dataset_size) * 100
        print(f"{word}: Estimated Absolute Frequency = {estimated_absolute_frequency:.0f}, "
              f"Estimated Relative Frequency = {estimated_relative_frequency:.2f}%")

<font size="+1" color="red">Replace this cell with a brief commentary indicating what reservoir size you would recommend to use, and your overall conclusions.</font>

For small sizes (50, 100) we get inconsistent results between runs, with size 500 there is some more consistency but there are still some fluctuations. For larger sizes (1000, 1500) we see more stable results, getting in 2 consecutive runs the same top-3 words (and in 1 case, size 100, same top-5 with a slight difference in frequencies). 

Given the above observations, a reservoir size of at least 1000 is recommended as these sizes yield more stable results, with consistent top words across trials, which seems that as we increase the size we get more stable results.

# 2. Determine approximately the distinct number of words

In [ ]:
# Leave this code as-is

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [ ]:
# Leave this code as-is

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

<font size="+1" color="red">Replace this cell with your code to perform the requested number of passes.</font>

In [ ]:
number_of_passes = 5
estimates = []

for i in range(number_of_passes):
    hash_function = random_hash_function()
    
    max_trailing_zeroes = 0
    
    for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words=20000, report_every=5000):
        hash_value = hash_function(word) 
        trailing_zeroes = count_trailing_zeroes(hash_value) 
        max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes) 
        
    estimate = 2 ** max_trailing_zeroes
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i+1, estimate))



In [ ]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

<font size="+1" color="red">Compute the median of average estimates in 3 separate runs of your algorithm; each run should do 10 passes over the file. Repeat this for nouns (POS_NOUN), adjectives (POS_ADJECTIVE), and verbs (POS_VERB). Replace this cell with the results you obtained in each pass, and whether the average or the median seem more appropriate for this probabilistic counting.</font>

In [ ]:
def flajolet_martin_estimate(pos_tag, num_runs=3, num_passes=10):
    all_run_estimates = []

    for run in range(num_runs):
        print(f"Run {run + 1} for POS Tag: {pos_tag}")
        estimates = []

        for pass_idx in range(num_passes):
            hash_function = random_hash_function()

            max_trailing_zeroes = 0

            for word in read_by_parts_of_speech(INPUT_FILE, [pos_tag], max_words=5000, report_every=1000):
                hash_value = hash_function(word) 
                trailing_zeroes = count_trailing_zeroes(hash_value)  
                max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes)  

            estimate = 2 ** max_trailing_zeroes
            estimates.append(estimate)
            print(f"  Pass {pass_idx + 1}: Estimate = {estimate}")

        median_estimate = statistics.median(estimates)
        print(f"Median for Run {run + 1}: {median_estimate}")
        all_run_estimates.append(median_estimate)

    overall_median = statistics.median(all_run_estimates)
    print(f"\nOverall Median for POS {pos_tag}: {overall_median}\n")
    return all_run_estimates, overall_median


noun_results, noun_median = flajolet_martin_estimate(POS_NOUN)
adjective_results, adjective_median = flajolet_martin_estimate(POS_ADJECTIVE)
verb_results, verb_median = flajolet_martin_estimate(POS_VERB)

Flajolet-Martin is a probabilistic method that can produce high variance in estimates; the median mitigates this by focusing on the central tendency of the values.
Hence, for this problem the median is more appropriate as it robustly accounts for the probabilistic nature of the algorithm and its potential variance. This is particularly evident when certain runs yield anomalously high or low estimates.

## EXTRA POINTS

In [ ]:
import matplotlib.pyplot as plt
import statistics

def heaps_law_experiment(pos_tag, max_words_list, num_passes=3):
    results = []  # Store (max_words, distinct_words) pairs

    for max_words in max_words_list:
        estimates = []

        for _ in range(num_passes):
            hash_function = random_hash_function()  # Generate a new hash function
            max_trailing_zeroes = 0

            for word in read_by_parts_of_speech(INPUT_FILE, [pos_tag], max_words=max_words, report_every=50000):
                hash_value = hash_function(word)  
                trailing_zeroes = count_trailing_zeroes(hash_value)  
                max_trailing_zeroes = max(max_trailing_zeroes, trailing_zeroes) 

            estimate = 2 ** max_trailing_zeroes
            estimates.append(estimate)

        median_estimate = statistics.median(estimates)  
        results.append((max_words, median_estimate))
        print(f"max_words: {max_words}, Distinct Words (Median Estimate): {median_estimate}")

    return results


def plot_heaps_law(results, pos_tag):
    """Plot the results of the Heap's Law experiment."""
    max_words, distinct_words = zip(*results)
    plt.figure(figsize=(10, 6))
    plt.plot(max_words, distinct_words, marker='o', label=pos_tag)
    plt.xlabel("Total Words Read")
    plt.ylabel("Distinct Words (Estimated)")
    plt.title(f"Heap's Law for {pos_tag}")
    plt.legend()
    plt.grid(True)
    plt.show()


max_words_list = [1000, 2000, 4000, 8000, 16000, 32000]  # Logarithmic intervals
num_passes = 3  

print("Processing nouns...")
noun_results = heaps_law_experiment(POS_NOUN, max_words_list, num_passes=num_passes)

print("Processing adjectives...")
adjective_results = heaps_law_experiment(POS_ADJECTIVE, max_words_list, num_passes=num_passes)

print("Processing verbs...")
verb_results = heaps_law_experiment(POS_VERB, max_words_list, num_passes=num_passes)

plot_heaps_law(noun_results, "Nouns")
plot_heaps_law(adjective_results, "Adjectives")
plot_heaps_law(verb_results, "Verbs")


<font size="+2" color="#003300">I hereby declare that, except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>